Aggregate image color features to restaurants and build restaurant level models

In [123]:
import pandas as pd
import numpy as np

In [124]:
color_dir = '/color/F3_all_train.npy'
bus_dir = '/pic_label_subbybus_train.csv'
color_test = '/color/F3_testnew.npy'
bus_test_dir = '/pic_label_subbybus_test.csv'

In [125]:
color = np.load(color_dir)
color_test = np.load(color_test)
bus_info = pd.read_csv(bus_dir, header = 0)
colordf = pd.DataFrame(color)
bus_info_test = pd.read_csv(bus_test_dir, header = 0)
color_testdf = pd.DataFrame(color_test)

In [126]:
bus_photoid = bus_info[['photo_id','business_id']]
bus_test_photoid = bus_info_test[['photo_id','business_id']]
print bus_test_photoid[0:5]

   photo_id  business_id
0        80         1114
1        95         3065
2       130         2687
3       143         3979
4       296         1114


In [127]:
color_withid = colordf.join(bus_photoid)
color_withid_test = color_testdf.join(bus_test_photoid)

method 1: color_count/number_photo

In [128]:
color_withid[['photo_id']] =1
color_withid_test[['photo_id']] =1
print color_withid[0:5]

   0  1  2  3  4  5  6  7  8  9     ...       8992  8993  8994  8995  8996  \
0  1  0  0  1  1  1  1  1  1  1     ...          0     0     1     0     0   
1  0  0  0  0  0  0  0  0  0  0     ...          0     0     1     1     0   
2  0  0  0  0  0  0  0  0  0  0     ...          0     0     0     0     0   
3  1  1  1  1  0  0  1  1  1  0     ...          0     0     1     0     0   
4  0  1  1  0  0  0  0  1  1  0     ...          0     0     0     0     0   

   8997  8998  8999  photo_id  business_id  
0     0     0     0         1         3305  
1     0     0     0         1          242  
2     0     0     0         1         1202  
3     0     0     0         1         3799  
4     0     0     0         1          276  

[5 rows x 9002 columns]


In [129]:
sumdf_train= color_withid.groupby(['business_id']).sum().reset_index()

sumdf_test = color_withid_test.groupby(['business_id']).sum().reset_index()

In [130]:
print sumdf_train[0:4]

   business_id   0   1   2   3   4   5   6   7   8    ...     8991  8992  \
0            7  10   8  11  11  10   9   8   8   9    ...        3     2   
1            8   9  12  10  11  13  10   7  13  15    ...        0     0   
2           23  20  28  35  31  39  35  32  40  43    ...        9     9   
3           28   8   3   4   3   0   1   8   5   4    ...        1     0   

   8993  8994  8995  8996  8997  8998  8999  photo_id  
0     1     2     4     4     3     2     1        21  
1     0     4     1     0     0     0     0        34  
2     3    32    10     7     7     7     4        90  
3     0     6     3     0     0     0     0        16  

[4 rows x 9002 columns]


In [131]:
for i in range(9000):
    sumdf_train[i] = sumdf_train.apply(lambda row: row[i]*1.0/row['photo_id'], axis=1)
    sumdf_test[i] = sumdf_test.apply(lambda row: row[i]*1.0/row['photo_id'], axis=1)

In [111]:
sumdf_train.drop(['photo_id'],inplace=True,axis=1)
sumdf_test.drop(['photo_id'],inplace=True,axis=1)
print sumdf_train[0:3]

   business_id         0         1         2         3         4         5  \
0            7  0.904762  0.476190  0.761905  0.904762  0.904762  0.904762   
1            8  0.617647  0.588235  0.588235  0.617647  0.794118  0.794118   
2           23  0.833333  0.744444  0.855556  0.855556  0.888889  0.955556   

          6         7         8    ...          990       991       992  \
0  0.904762  0.857143  0.857143    ...     0.571429  0.761905  0.571429   
1  0.882353  0.852941  0.852941    ...     0.617647  0.500000  0.441176   
2  0.977778  0.966667  0.966667    ...     0.588889  0.577778  0.477778   

        993       994       995       996       997       998       999  
0  0.619048  0.571429  0.571429  0.476190  0.333333  0.285714  0.095238  
1  0.323529  0.205882  0.176471  0.029412  0.029412  0.000000  0.000000  
2  0.377778  0.333333  0.244444  0.222222  0.211111  0.144444  0.088889  

[3 rows x 1001 columns]


In [112]:
bus_id = bus_info.drop_duplicates(cols='business_id', take_last=True)
bus_id_test = bus_info_test.drop_duplicates(cols='business_id', take_last=True)
#bus_id.to_csv('bus_label_train.csv',index=False)
#bus_id_test.to_csv('bus_label_test.csv',index=False)

/Users/s92wang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
  if __name__ == '__main__':
/Users/s92wang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead
  if __name__ == '__main__':
/Users/s92wang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
  from ipykernel import kernelapp as app
/Users/s92wang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead
  from ipykernel import kernelapp as app


In [113]:
total_train = pd.merge(sumdf_train,bus_id,on='business_id', how='left')
total_test = pd.merge(sumdf_test,bus_id_test,on='business_id', how='left')

In [114]:
print total_test[0:5]

   business_id         0         1         2         3         4         5  \
0           18  0.250000  0.375000  0.500000  0.625000  0.875000  0.750000   
1           38  0.403101  0.457364  0.519380  0.612403  0.705426  0.689922   
2           96  0.584615  0.523077  0.615385  0.569231  0.569231  0.615385   
3          112  0.365591  0.365591  0.440860  0.494624  0.505376  0.537634   
4          221  0.206349  0.126984  0.190476  0.248677  0.264550  0.296296   

          6         7         8   ...     photo_id  label_0  label_1  label_2  \
0  0.500000  0.375000  0.125000   ...       373017        0        1        1   
1  0.720930  0.697674  0.348837   ...       470463        0        0        0   
2  0.584615  0.584615  0.569231   ...       460707        1        0        0   
3  0.473118  0.419355  0.311828   ...       470754        0        1        1   
4  0.259259  0.232804  0.174603   ...       466859        0        1        1   

   label_3  label_4  label_5  label_6  label

In [115]:
train_x = total_train.iloc[:,1:5761]
test_x = total_test.iloc[:,1:5761]


In [116]:
from sklearn import linear_model
from sklearn import metrics

In [117]:
prediction = pd.DataFrame(total_test['business_id'])
print prediction[0:5]

   business_id
0           18
1           38
2           96
3          112
4          221


In [118]:
for i in range(8):
    train_y = total_train['label_%d' % i]
    test_y = total_test['label_%d' % i]
    logreg = linear_model.LogisticRegression()
    logreg.fit(train_x, train_y)
    pred_y = logreg.predict(test_x)
    pred_y_pd = pd.DataFrame(pred_y, columns = ['label_%d' % i])
    prediction = prediction.join(pred_y_pd)
    accuracy = metrics.accuracy_score(pred_y, test_y)
    f1 = metrics.f1_score(pred_y, test_y)
    print 'label_%d' % i, accuracy,f1
    

label_0 0.757575757576 0.647058823529
label_1 0.767676767677 0.792792792793
label_2 0.79797979798 0.818181818182
label_3 0.545454545455 0.545454545455
label_4 0.808080808081 0.716417910448
label_5 0.717171717172 0.791044776119
label_6 0.808080808081 0.867132867133
label_7 0.858585858586 0.810810810811


In [119]:
print prediction[0:5]

   business_id  label_0  label_1  label_2  label_3  label_4  label_5  label_6  \
0           18        1        0        0        0        0        0        0   
1           38        1        0        0        1        0        1        1   
2           96        0        1        1        1        1        1        1   
3          112        0        1        1        0        1        1        1   
4          221        0        1        1        1        1        1        1   

   label_7  
0        0  
1        0  
2        0  
3        1  
4        1  


In [120]:
# score function
def mlb_f1(y_true, y_pred):
    y_true = set(y_true)
    y_pred = set(y_pred)
    tp = len(y_true & y_pred)
    fp = len(y_pred-y_true)
    fn = len(y_true-y_pred)
    if tp == 0 and fp == 0:
        return 0.0
    else:
        p = float(tp)/(tp+fp)
        r = float(tp)/(tp+fn)
        return 2*p*r/(p+r) if tp!=0 else 0.
def mean_f1(y_true, y_pred):
    score_list = map(lambda x: mlb_f1(x[0],x[1]), zip(y_true, y_pred))
    return sum(score_list)/len(score_list)

In [121]:
y_pred = [[i for i in range(len(arr)) if arr[i]!=0] for arr in np.array(prediction.iloc[:,1:])]
true_bus = total_test.loc[:, 'label_0':'label_8']
y_true = [[i for i in range(len(arr)) if arr[i]!=0] for arr in np.array(true_bus)]
print y_true[0:5]
print y_pred[0:5]
print prediction[0:5]

[[1, 2, 3, 4, 5, 6, 7], [5, 6, 8], [0, 3, 5, 6, 8], [1, 2, 4, 5, 6, 7], [1, 2, 4, 5, 6, 7]]
[[0], [0, 3, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7]]
   business_id  label_0  label_1  label_2  label_3  label_4  label_5  label_6  \
0           18        1        0        0        0        0        0        0   
1           38        1        0        0        1        0        1        1   
2           96        0        1        1        1        1        1        1   
3          112        0        1        1        0        1        1        1   
4          221        0        1        1        1        1        1        1   

   label_7  
0        0  
1        0  
2        0  
3        1  
4        1  


In [122]:
mean_f1(y_true, y_pred)

0.6221605554938885